In [ ]:
# import settings and functions
%run ./../../imports.ipynb


In [ ]:
csv_name = "dummy_tree_test.csv"

exe = hdiv_data_driven_diffusion_snes

In [ ]:
!cp ./../comparison_hat_DD/param_file.petsc ./param_file.petsc
!cp ./../comparison_hat_DD/dummy_tree_test.csv ./dummy_tree_test.csv
!cp ./../comparison_hat_DD/scaling.in ./scaling.in

In [ ]:
# # Change according to instruction above
analytical_solution_tag = "-ana_mexi_hat"
generate_config = generateConfig_squareMexiHat
generate_mesh = generateMesh_squareMexiHat

In [ ]:
prefix = "c5_hat_missing_"

params.conductivity = 1.0 # linear conductivity
# params.element_size = elem_size_list[0] # element size in the regular mesh
params.element_size = 0.1
params.order = 1 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "square_mexi"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]


In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
params.show_mesh = True
generate_config(params)
generate_mesh(params)

In [ ]:
params.nproc = 1
params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1

In [ ]:
realisations = 30
run_analysis = True

ana_csv_name = "test"

params.order = 1

ref_parts = "-refinement_style 1 -ref_iter_num 6 -tol_distance_ref 4.0 "

refinement_style = 1


if run_analysis:
    !rm sumanalys.csv # making sure the previous analysis doesn't affect this one

    for i in range(realisations):
        !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {csv_name}-write_long_error_file -rand_ini {ref_parts}

    !mv ./sumanalys.csv ./{ana_csv_name}.csv


## load and plot results

In [ ]:
data = pd.read_csv(f'{ana_csv_name}.csv', header=None, names=naming,  index_col=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_dd_shaded_ana(ana_df_list, noise_list, errorType='L2norm', label="", yscale_log=False):
    fig = plt.figure()
    plot_title = 'Mean with Standard Deviation'

    for a in range(0, len(ana_df_list)):
        # Group by 'datanum' and calculate mean and std
        grouped_data = ana_df_list[a].groupby(['datanum']).agg([np.mean, np.std])
        x = np.array(grouped_data.index).astype(float)
        y_mean = np.array(grouped_data[(errorType, 'mean')]).astype(float)
        y_std = np.array(grouped_data[(errorType, 'std')]).astype(float)

        # Plot mean
        plt.plot(x, y_mean, '-x', label='noise = ' + str(noise_list[a]))

        # Shade the area for standard deviation
        plt.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.5)

    plt.xscale('log')
    if yscale_log:
        plt.yscale('log')
    plt.legend(loc='best')
    if label:
        plt.ylabel(label)
    else:
        plt.ylabel(errorType)
    plt.xlabel(r'\# data points')
    plt.title(plot_title)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(errorType + '.png')
    plt.show()
    plt.close()